# Выбор локации для скважины

Компания «ГлавРосГосНефть» занимается добычей нефти. Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

* Бюджет на разработку скважин в регионе — 10 млрд рублей.

* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [ ]:
# Импортируем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from tqdm import tqdm

In [ ]:
# откроем данные
data_a = pd.read_csv('/datasets/geo_data_0.csv')
data_b = pd.read_csv('/datasets/geo_data_1.csv')
data_c = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
# изучим данные
data_a.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


* **id** — уникальный идентификатор скважины;
* **f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);
* **product** — объём запасов в скважине (тыс. баррелей).

In [ ]:
data_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В данных отсутствуют пропуски.
Столбец id ябляется избыточной информацией, поэтому его нужно удалить.

In [ ]:
# напишем функцию для удаления столбца 'id'
def drop_id(data):
    df = data.drop('id', axis=1)
    return df

In [ ]:
# удалим столбец 'id' из всех трех датасетов
data_a = drop_id(data_a)
data_b = drop_id(data_b)
data_c= drop_id(data_c)

In [ ]:
data_a.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


### Выводы

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

* В данных не обнаружено пропусков.
* Из всех трех датасетов были удалены столбцы 'id'

## Обучение и проверка модели

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

In [ ]:
# Напишем функцию, которая будет разбивать данные в заданной пропорции
def data_separation(data):
    features = data.drop('product', axis=1)
    target = data['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [ ]:
# Используем функцию data_separation для разделения данных на выборки
features_train_a, features_valid_a, target_train_a, target_valid_a = data_separation(data_a)
features_train_b, features_valid_b, target_train_b, target_valid_b = data_separation(data_b)
features_train_c, features_valid_c, target_train_c, target_valid_c = data_separation(data_c)

Обучим модель и сделаем предсказания на валидационной выборке. Сохраним предсказания и правильные ответы на валидационной выборке. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [ ]:
data = [data_a, data_b, data_c]

features_train = [features_train_a, features_train_b, features_train_c]
features_valid = [features_valid_a, features_valid_b, features_valid_c]
target_train = [target_train_a, target_train_b, target_train_c]
target_valid = [target_valid_a, target_valid_b, target_valid_c]

regions = ['Регион a', 'Регион b', 'Регион c']

In [ ]:
predictions = []
targets = []

for i in range(len(data)):
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])

    predictions.append(model.predict(features_valid[i]))
    targets.append(target_valid[i])

    print (regions[i])
    print ('Средний запас предсказанного сырья: ', round(predictions[i].mean(), 1), 'тыс. баррелей')
    print ('RMSE модели: ', round((mean_squared_error(targets[i], predictions[i]) ** 0.5), 1), 'тыс. баррелей')
    print (' ')

Регион a
Средний запас предсказанного сырья:  92.6 тыс. баррелей
RMSE модели:  37.6 тыс. баррелей
 
Регион b
Средний запас предсказанного сырья:  68.7 тыс. баррелей
RMSE модели:  0.9 тыс. баррелей
 
Регион c
Средний запас предсказанного сырья:  95.0 тыс. баррелей
RMSE модели:  40.0 тыс. баррелей
 


### Выводы

Была обучена модель LinearRegression и сделаны предсказания на валидационной выборке.
Расчитан средний запас предсказанного сырья и RMSE модели для каждого из регионов. Наиболее перспективным местом с точки зрения величины запаса предсказанного сырья является регион с (Средний запас предсказанного сырья:  95.0 тыс. баррелей
RMSE модели:  40.0 тыс. баррелей).

## Подготовка к расчёту прибыли

Сохраним ключевые значения для расчетов в отдельных переменных.

In [ ]:
# точки для исследования при разведке региона
RESEARCH_POINTS = 500

# лучшие точки для разработки в соответствии с результатами машинного обучения
BEST_ML_POINTS = 200

# бюджет на разработку скважин в регионе
BUDGET = 10000000000

# доход с одного барреля сырья
REVENUE = 450000

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [ ]:
min_volume = BUDGET / BEST_ML_POINTS / REVENUE

print ('Минимальный объем сырья для безубыточной разработки скважины: ', round(min_volume), 'тыс. баррелей')

Минимальный объем сырья для безубыточной разработки скважины:  111 тыс. баррелей


### Выводы

Было показано, что достаточный объём сырья для безубыточной разработки новой скважины составляет 111 тыс. баррелей.

Ранее было обнаружено, что даже самый прибыльный регион с имеет запас около 95 тыс. баррелей. Это на 16 тыс. баррелей меньше необходимого.

## Расчёт прибыли по выбранным скважинам и предсказаниям модели

In [ ]:
# Подготовим данные для расчета

# для региона а
target_a = pd.Series(targets[0]).reset_index(drop=True)
prediction_a = pd.Series(predictions[0])

# для региона b
target_b = pd.Series(targets[1]).reset_index(drop=True)
prediction_b = pd.Series(predictions[1])

# для региона c
target_c = pd.Series(targets[2]).reset_index(drop=True)
prediction_c = pd.Series(predictions[2])

In [ ]:
# Напишем функцию для расчёта прибыли
def revenue_counter(target, prediction, count):

    # выберем скважины с максимальными значениями предсказаний
    sorted_prediction = prediction.sort_values(ascending=False)
    aim = target[sorted_prediction.index][:count]

    # функция вернет суммарную прибыль по выбранным скважинам
    return (REVENUE * aim.sum()) - BUDGET


In [ ]:
# Посчитаем прибыль для каждого региона

# для региона а
print ('Суммарная прибыль в регионе a: ', round(
    revenue_counter(target_a, prediction_a, BEST_ML_POINTS)), 'тыс. р')
print (' ')

# для региона b
print ('Суммарная прибыль в регионе b: ', round(
    revenue_counter(target_b, prediction_b, BEST_ML_POINTS)), 'тыс. р')
print (' ')

# для региона c
print ('Суммарная прибыль в регионе c: ', round(
    revenue_counter(target_c, prediction_c, BEST_ML_POINTS)), 'тыс. р')
print (' ')

Суммарная прибыль в регионе a:  3320826043 тыс. р
 
Суммарная прибыль в регионе b:  2415086697 тыс. р
 
Суммарная прибыль в регионе c:  2710349964 тыс. р
 


### Выводы

Были расчётаны прибыли по выбранным скважинам и предсказаниям модели. Оказалось, что большую прибыль может принести скважины в  районе а: 3320826043 тыс. р

## Расчёт прибыли и рисков

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [ ]:
%%time

# создадим списки для трех регионов
target = [target_a, target_b, target_c]
predictions = [prediction_a, prediction_b, prediction_c]
regions = ['Регион a', 'Регион b', 'Регион c']

state = np.random.RandomState(12345)

for i in range(len(target)):
    values = []

    for j in range (1000):
        target_subsample = target[i].sample(RESEARCH_POINTS, replace=True, random_state=state)
        predicted_subsample = predictions[i].loc[target_subsample.index]


        # расчет дохода с 200 самых прибыльных
        values.append(revenue_counter(target_subsample, predicted_subsample, BEST_ML_POINTS))

    values = pd.Series(values)

    # расчет средней прибыли и доверительного интервала
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

     # расчет риска убытков
    #risk = (values < BUDGET).mean()
    risk = values[values < 0].count()/values.count()


    print (regions[i])
    print ('Средняя прибыль: ', round((mean / 1000000000), 3), 'млрд. р')
    print ('95% доверительный интервал: от ', round((lower / 1000000000), 3), 'млрд. р до ',
           round((upper / 1000000000), 3), 'млрд. р')
    print ('Риск убытков: {:.1%}'.format (risk))
    print (' ')


Регион a
Средняя прибыль:  0.426 млрд. р
95% доверительный интервал: от  -0.102 млрд. р до  0.948 млрд. р
Риск убытков: 6.0%
 
Регион b
Средняя прибыль:  0.518 млрд. р
95% доверительный интервал: от  0.128 млрд. р до  0.954 млрд. р
Риск убытков: 0.3%
 
Регион c
Средняя прибыль:  0.42 млрд. р
95% доверительный интервал: от  -0.116 млрд. р до  0.99 млрд. р
Риск убытков: 6.2%
 
CPU times: user 4.25 s, sys: 18.5 ms, total: 4.27 s
Wall time: 4.53 s


### Выводы

Анализ данных показал, что абсолютным лидером является регион b (0,518 млрд. р). Несмотря на то что все 3 региона показывают сопоставимую прибыль, риск убытков в регионе b в ~20 раз ниже (0,3%).

## Выводы

Для добывающей компании  «ГлавРосГосНефть» нужно решить в каком регионе бурить новую скважину. Были предоставлены данные о трех регионах, на основании которых была построена модель для определения наиболее первспективного региона.

* Данные были загружены и подготовлены, после чего разбиты на обучающую и валидационную выборки в соотношении 75:25.
* Была обучена модель LinearRegression и сделаны предсказания на валидационной выборке.
* Расчитан средний запас предсказанного сырья и RMSE модели для каждого из регионов. Наиболее перспективным местом с точки зрения величины запаса предсказанного сырья является регион с (Средний запас предсказанного сырья:  95.0 тыс. баррелей RMSE модели:  40.0 тыс. баррелей).
* Было показано, что достаточный объём сырья для безубыточной разработки новой скважины составляет 111 тыс. баррелей. Ранее было обнаружено, что даже самый прибыльный регион с имеет запас около 95 тыс. баррелей. Это на 16 тыс. баррелей меньше необходимого.
* Были расчётаны прибыли по выбранным скважинам и предсказаниям модели. Оказалось, что большую прибыль может принести скважины в районе а: 3.320 млрд. р
* Анализ данных показал, что абсолютным лидером является регион b (0,518 млрд. р). Несмотря на то что все 3 региона показывают сопоставимую прибыль, риск убытков в регионе b в ~20 раз ниже (0,3%).